#### TEST Model Running

In [1]:
import ollama

response = ollama.chat(
    model="llama3:8b",   # เร็วกว่า deepseek
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant. Answer briefly and clearly."
        },
        {
            "role": "user",
            "content": "Explain RAG in simple terms"
        }
    ]
)

print(response["message"]["content"])


RAG stands for "Red, Amber, Green" and is a simple way to categorize project status or progress. It's often used in Agile project management.

* Red (R): Something's not going well, there are issues or problems.
* Amber (A): There are some concerns or things to watch out for, but it's still manageable.
* Green (G): Everything is going smoothly and on track!

It helps teams quickly see the status of their projects and make decisions accordingly.


### START 

In [2]:
import os
import streamlit as st
from datetime import date
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA

### PART 1 : CONFIG (ตั้งค่าระบบ)

In [3]:
PERSIST_DIR = "./chroma_db_promotions"
COLLECTION_NAME = "promotions"

EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
LLM_MODEL = "llama3"

### PART 2 : SAMPLE DATA 

In [4]:
SAMPLE_PROMOS = [
    {
        "doc_id": "promo_101_p01",
        "product_id": 101,
        "product_sku": "TSR20",
        "product_name": "Rubber TSR20",
        "category": "TSR",
        "country": "Thailand",
        "promotion_id": "P01",
        "promotion_name": "Heavy Discount 3 Days",
        "promotion_detail": "Heavy discount for 3 days. Target: export customers. Condition: minimum order 10 tons.",
        "discount_type": "percentage",
        "discount_value": 15,
        "duration_days": 3,
        "start_date": "2026-01-25",
        "end_date": "2026-01-27",
        "channel": "B2B",
        "status": "active",
        "priority": 90,
    },
    {
        "doc_id": "promo_101_p02",
        "product_id": 101,
        "product_sku": "TSR20",
        "product_name": "Rubber TSR20",
        "category": "TSR",
        "country": "Thailand",
        "promotion_id": "P02",
        "promotion_name": "Heavy Discount 7 Days",
        "promotion_detail": "Heavy discount for 7 days. Condition: contract customers only. Includes free logistics within Thailand.",
        "discount_type": "percentage",
        "discount_value": 10,
        "duration_days": 7,
        "start_date": "2026-01-25",
        "end_date": "2026-01-31",
        "channel": "B2B",
        "status": "active",
        "priority": 80,
    },
    {
        "doc_id": "promo_102_p03",
        "product_id": 102,
        "product_sku": "CUPLUMP",
        "product_name": "Cup Lump",
        "category": "RAW",
        "country": "Thailand",
        "promotion_id": "P03",
        "promotion_name": "Short Flash Deal 2 Days",
        "promotion_detail": "Flash deal for 2 days. Best price for farm pickup only.",
        "discount_type": "fixed",
        "discount_value": 500,
        "duration_days": 2,
        "start_date": "2026-01-26",
        "end_date": "2026-01-27",
        "channel": "B2C",
        "status": "active",
        "priority": 70,
    },
]


### PART 3 : Build_embedding_text 

In [5]:
def build_embedding_text(row: dict) -> str:
    return f"""
        Product Name: {row.get('product_name')}
        SKU: {row.get('product_sku')}
        Category: {row.get('category')}
        Country: {row.get('country')}

        Promotion Name: {row.get('promotion_name')}
        Promotion Detail: {row.get('promotion_detail')}
        Discount: {row.get('discount_type')} {row.get('discount_value')}
        Duration: {row.get('duration_days')} days
        Channel: {row.get('channel')}
        Status: {row.get('status')}
        """.strip()

### PART 4 : Build_metadata

In [6]:
def build_metadata(row: dict) -> dict:
    return {
        "doc_id": row["doc_id"],

        "product_id": row["product_id"],
        "product_sku": row["product_sku"],
        "product_name": row["product_name"],
        "category": row["category"],
        "country": row["country"],

        "promotion_id": row["promotion_id"],
        "promotion_name": row["promotion_name"],
        "duration_days": row["duration_days"],
        "discount_type": row["discount_type"],
        "discount_value": row["discount_value"],
        "start_date": row["start_date"],
        "end_date": row["end_date"],
        "channel": row["channel"],
        "status": row["status"],
        "priority": row["priority"],
    }

### PART 5 : Embedding + Chroma Init

In [7]:
@st.cache_resource
def get_embeddings():
    return HuggingFaceEmbeddings(model_name=EMBED_MODEL)

In [8]:
def load_or_create_chroma():
    embeddings = get_embeddings()
    vectordb = Chroma(
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIR,
        embedding_function=embeddings,
    )
    return vectordb

### PART 6 : Seed data

In [16]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")

# test
embeddings.embed_query("test")[0]


0.5249887704849243